In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import json

from torch.utils.data import Dataset,DataLoader
import torch
from torch import nn
from torch.nn.modules.loss import _WeightedLoss
import timm
import time
from tqdm import tqdm
import cv2
import numpy as np
#from torch.cuda.amp import autocast, GradScaler

In [3]:
from albumentations import Resize

In [4]:
test_fold = 0
img_size = 512
bs = 16

In [5]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

In [6]:
TRAIN_AUGS = Resize(img_size, img_size)
VAL_AUGS = Resize(img_size, img_size)

## Support Functions

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

## Read Data

In [8]:
data = pd.read_csv('train.csv')
data.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [9]:
len(data)
data.label.hist()

21397

In [10]:
mapping = json.load(open('label_num_to_disease_map.json', 'r'))
mapping

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [11]:
data['fold'] = 0
strkf = StratifiedKFold(n_splits=5)
strkf.get_n_splits(data.image_id, data.label)
f = 0
for train_index, test_index in strkf.split(data.image_id, data.label):
    #print("TRAIN:", train_index, "TEST:", test_index)
    data.loc[data.index.isin(test_index), 'fold'] = f 
    f+=1

5

In [12]:
data.fold.value_counts()

1    4280
0    4280
4    4279
3    4279
2    4279
Name: fold, dtype: int64

In [13]:
train_data = data[data.fold != test_fold].reset_index(drop=True)
val_data = data[data.fold == test_fold].reset_index(drop=True)

## Cassava dataset

In [14]:
class CassavaDataset(Dataset):
    def __init__(self, df, val = False):
        super().__init__()
        self.df = df
        self.val = val
        
    def __len__(self, ):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df[self.df.index == idx]
        #print)
        image_name = row.image_id.values[0]
        img = get_img('train_images/'+ image_name)
        if not self.val:
            label = row.label.values[0]
            img = TRAIN_AUGS(image=img)['image']
            img = np.moveaxis(img, 2, 0)
            return img, label
        img = VAL_AUGS(image=img)['image']
        return img

In [15]:
train_ds = CassavaDataset(train_data, val = False)
valid_ds = CassavaDataset(val_data, val = False)

In [16]:
train_ds[0][0].shape

(3, 512, 512)

In [17]:
train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=bs,
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=8,
        #sampler=BalanceClassSampler(labels=train_['label'].values, mode="downsampling")
    )
val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=bs*2,
        num_workers=8,
        shuffle=False,
        pin_memory=False,
)

In [18]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        #print(image_labels.shape, exam_label.shape)
        #with autocast():
        image_preds = model(imgs)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)

        loss = loss_fn(image_preds, image_labels)
        loss.backward()
        #scaler.scale(loss).backward()

        if running_loss is None:
            running_loss = loss.item()
        else:
            running_loss = running_loss * .99 + loss.item() * .01

        if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
            # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)
            optimizer.step()
            #scaler.step(optimizer)
            #scaler.update()
            optimizer.zero_grad() 

            if scheduler is not None and schd_batch_update:
                scheduler.step()

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
            description = f'epoch {epoch} loss: {running_loss:.4f}'

            pbar.set_description(description)

    if scheduler is not None and not schd_batch_update:
        scheduler.step()
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

In [19]:
# reference: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/173733
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [20]:
class CassvaImgClassifier(torch.nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
device = torch.device('cuda')
model = CassvaImgClassifier('tf_efficientnet_b4_ns', data.label.nunique(), pretrained=True).to(device)
#scaler = GradScaler()  
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
 
loss_tr = nn.CrossEntropyLoss().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
for epoch in range(10):
    train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)
    with torch.no_grad():
        valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)

    torch.save(model.state_dict(),'{}_fold_{}_{}'.format(CFG['model_arch'], test_fold, epoch))
            
del model, optimizer, train_loader, val_loader, scaler, scheduler
torch.cuda.empty_cache()

epoch 0 loss: 0.3739: 100%|██████████| 134/134 [00:30<00:00,  4.40it/s]


validation multi-class accuracy = 0.8720


epoch 1 loss: 0.2728:  51%|█████▏    | 550/1070 [04:20<04:07,  2.10it/s]